 # Projet web Mining : 
 # FEUDJIO NGOUANET VALDES VOTEL
 # LAGNOL    

### PARTIE 1 : WEB SCRAPING DE LA BASE DE DONNEES

#### installation des packages de base

In [62]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json
!pip install feedparser
!pip install newspaper3k
!python -m spacy download fr_core_news_md

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/45.8 MB ? eta -:--:--
     -- ------------------------------------- 2.4/45.8 MB 14.9 MB/s eta 0:00:03
     ---- ----------------------------------- 5.2/45.8 MB 13.8 MB/s eta 0:00:03
     ------ --------------------------------- 7.6/45.8 MB 12.7 MB/s eta 0:00:04
     -------- ------------------------------ 10.0/45.8 MB 12.4 MB/s eta 0:00:03
     ---------- ---------------------------- 12.6/45.8 MB 12.3 MB/s eta 0:00:03
     ------------ -------------------------- 14.7/45.8 MB 12.1 MB/s eta 0:00:03
     ------------- ------------------------- 16.3/45.8 MB 11.5 MB/s eta 0:00:03
     -------------- ------------------------ 16.8/45.8 MB 11.0 MB/s eta 0:00:03
     --------------- ------------------------ 17.3/45.8 MB 9.8 MB/s eta 0:00:03
     --------------- ------------------------ 17.8/45.8 MB 9.0 MB/s eta 0:00:04
     ---------------- ----------------------- 18.6/45.8 MB 8.2 MB/s eta 0:00:04
     ---------------- ----------------------- 19


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
import json

with open('tvinfo-sources.json','r') as f1:
    tvinfo=json.load(f1)
    
with open('francetvinfo.json.','r') as f2:
    francetvinfo=json.load(f2)

#### Chargement des données deja existantes et des données sur les flus RSS

In [64]:
import json
import ssl
import feedparser as fp

# (a) Se prémunir contre le blocage de commande
ssl._create_default_https_context = ssl._create_unverified_context

# (b) Lire la base de données existante (francetvinfo.json)
with open("francetvinfo.json", "r", encoding="utf-8") as f:
    existing_data = json.load(f)


# (c) Lire les flux RSS listés dans tvinfo-sources.json
with open("tvinfo-sources.json", "r", encoding="utf-8") as f:
    rss_sources = json.load(f)


#### Selection des données qui ne sont pas encore dans la base de données existante

In [65]:
from newspaper import Article

# Extraire les liens existants pour éviter les doublons
existing_links = existing_data.keys()

# Scanner les flux RSS et repérer les nouveaux articles
new_articles = {}

for category, url in rss_sources.items():
    print(f"Lecture du flux RSS : {url}")
    data = fp.parse(url)

    # Parcourir les articles du flux RSS
    for item in data.entries:
        if item.link not in existing_links:  # Vérifier si l'article est nouveau
            try:
                # Utiliser newspaper pour extraire les informations détaillées
                article = Article(item.link)
                article.download()
                article.parse()

                # Ajouter les détails de l'article dans new_articles
                new_articles[item.link] = {
                    "title": article.title or item.title,
                    "date": article.publish_date if article.publish_date else (item.published if hasattr(item, "published") else "N/A"),
                    "author": article.authors or item.get("author", "Inconnu"),
                    "category": category,
                    "content": article.text or item.get("description", "Pas de contenu"),
                    "image_link": article.top_image or "Pas d'image"
                }
            except Exception as e:
                print(f"Erreur lors du traitement de l'article : {item.link}\n{e}")

print(f"{len(new_articles)} nouveaux articles ajoutés :")


Lecture du flux RSS : https://www.francetvinfo.fr/france.rss
Lecture du flux RSS : https://www.francetvinfo.fr/monde/europe.rss
Lecture du flux RSS : https://www.francetvinfo.fr/economie/entreprises.rss
58 nouveaux articles ajoutés :


In [66]:
# On affiche les liens des nouveaux articles parus das la base de données
for key in new_articles.keys():
    print(key)

https://www.francetvinfo.fr/replay-radio/aujourd-hui-c-est-demain/cyclone-chido-a-mayotte-les-alternatives-pour-produire-de-l-electricite-dans-les-zones-devastees_6934427.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/faits-divers/les-affrontements-entre-bandes-sont-en-legere-hausse-dans-l-agglomeration-parisienne-selon-le-prefet-de-police_6962687.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/replay-jt/franceinfo/21h-minuit/23-heures/paris-un-adolescent-poignarde-a-mort-lors-d-une-rixe_6962636.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/faits-divers/un-homme-de-20-ans-tue-par-balle-a-aix-en-provence-quatre-suspects-en-fuite_6962552.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/meteo/cyclone-ouragan/cyclone-chido-a-mayotte/cyclone-chido-a-mayotte-face-aux-reseaux-de-telecommunication-rompus-des-familles-sans-nouvelles-de-leurs-proches-se-tournent-vers-les-reseaux-sociaux_6962033.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/replay-jt/franceinfo/2

In [67]:
# faire une nouvelle base de données en y ajoutants de nouvelles données

francetvinfo_total=existing_data.update(new_articles)
len(existing_data)

2155

### Extraction d'informations

In [68]:
new_articles

{'https://www.francetvinfo.fr/replay-radio/aujourd-hui-c-est-demain/cyclone-chido-a-mayotte-les-alternatives-pour-produire-de-l-electricite-dans-les-zones-devastees_6934427.html#xtor=RSS-3-[general]': {'title': "Cyclone Chido à Mayotte : les alternatives pour produire de l'électricité dans les zones dévastées",
  'date': datetime.datetime(2024, 12, 18, 12, 54, 25, tzinfo=tzoffset(None, 3600)),
  'author': ['Anicet Mbida'],
  'category': 'france',
  'content': 'À Mayotte, dévastée par le cyclone Chido, l’électricité n’est que partiellement rétablie. Les groupes électrogènes, qui dépendent du diesel, sont difficiles à approvisionner, et les éoliennes classiques ne résistent pas aux vents violents. Quelles alternatives existent pour produire de l’électricité ?\n\nLes opérations de secours se poursuivent à Mayotte, dévastée par le cyclone Chido. L’électricité n’est, mercredi 18 décembre, que "partiellement remise en route", selon François-Noël Buffet, ministre des Outre-mer. Les pompiers e

In [69]:
import spacy
# load pipeline and related models
process = spacy.load("fr_core_news_md")
# process text with pipeline



In [70]:
commentaires = [new_articles[key]['content'] for key in new_articles.keys()]
commentaires[2]


'Un adolescent de 16 ans a été tué lors d\'une rixe entre bandes rivales à proximité d\'un lycée. La victime a été retrouvée inconsciente, poignardée par un coup de couteau.\n\nUn adolescent de 16 ans a été tué lors d\'une rixe entre bandes rivales à proximité d\'un lycée. La victime a été retrouvée inconsciente, poignardée par un coup de couteau.\n\nSur une vidéo amateur, filmée mardi 17 décembre, un jeune homme de 16 ans, au sol, est grièvement blessé à l\'arme blanche. Vers 8h30, devant un lycée du 13e arrondissement de Paris, une violente bagarre a éclaté entre adolescents. La victime a tenté d\'échapper à ses agresseurs avant de s\'écrouler quelques mètres plus loin. Le gardien d\'un immeuble lui a porté les premiers secours. "C\'est toujours dur de voir des enfants, comme ça, qui finissent à coups de couteau", confie Manuel Rodrigues.\n\nUn arrondissement touché par les rixes\n\nMalgré les secours, le jeune homme est décédé à l\'hôpital dans la matinée. Originaire du Val-de-Marne

In [71]:
import spacy
# load pipeline and related models
process = spacy.load("fr_core_news_md")
# list e that were detected
for element in commentaires :
    res = process(element)
    for e in res.ents:
        print(e.text, e.start, e.end, e.label_)

Mayotte 1 2 LOC
cyclone Chido 6 8 MISC
Mayotte 59 60 LOC
cyclone Chido 64 66 MISC
François-Noël Buffet 85 89 PER
Shall Energy 152 154 MISC
Portable 163 164 MISC
Mayotte 292 293 LOC
Paris 29 30 LOC
RMC 43 44 ORG
Laurent Nuñez 45 47 PER
Seine-Saint-Denis 69 70 LOC
Hauts-de-Seine 72 73 LOC
Val-de-Marne 75 76 LOC
Paris 97 98 LOC
parisiens 135 136 LOC
Amiral Mouchez 138 140 PER
Glacière" 141 142 MISC
Laurent Nuñez 176 178 PER
Paris 239 240 LOC
Laurent Nuñez 242 244 PER
13e arrondissement de Paris 107 111 LOC
Manuel Rodrigues 172 174 PER
Val-de-Marne 201 202 LOC
Reda Belhaj 255 257 PER
SGO Police Île-de-France 262 265 ORG
Paris 283 284 LOC
DCOS 28 29 ORG
Aix-en-Provence 53 54 LOC
Bouches-du-Rhône 55 56 LOC
France 63 64 LOC
franceinfo 112 113 ORG
Renault 129 130 ORG
Opel 138 139 ORG
Opel Corsa 180 182 MISC
Opel 207 208 ORG
France Bleu Provence 216 219 LOC
DCOS 262 263 ORG
cyclone Chido 3 5 MISC
océan Indien 13 15 LOC
cyclone Chido 75 77 MISC
Mayotte 78 79 LOC
Jessica 80 81 PER
Acoua 116 117 L

In [72]:
import spacy
from collections import Counter

# Charger le modèle spaCy en optimisant les composants nécessaires
nlp = spacy.load("fr_core_news_md", disable=["tagger", "parser"])  # Garder NER actif

# Liste contenant les contenus des articles et leurs liens
commentaires = {
    key: new_articles[key]['content']
    for key in new_articles.keys()
}

# Dictionnaire pour stocker les entités extraites par lien
resultats = {}

# Compteur global pour stocker les fréquences des entités
entite_counter = Counter()

# Utilisation de nlp.pipe pour traiter les articles efficacement
for key, doc in zip(commentaires.keys(), nlp.pipe(commentaires.values(), batch_size=20, n_process=2)):
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,   # Texte de l'entité
            "type": ent.label_,  # Type d'entité (PER, LOC, ORG, etc.)
            "start": ent.start_char,
            "end": ent.end_char
        })
        entite_counter[ent.text] += 1  # Compter la fréquence de l'entité globale

    # Stocker les entités pour chaque lien
    resultats[key] = {"entities": entities}

# Obtenir les 20 entités les plus représentées
top_20_entites = entite_counter.most_common(20)

# Affichage des résultats
print("Top 20 entités les plus représentées :")
print()
for ent, freq in top_20_entites:
    print(f"{ent} : {freq} occurrences")




Top 20 entités les plus représentées :

Mayotte : 37 occurrences
UE : 29 occurrences
Paris : 23 occurrences
Ukraine : 22 occurrences
France : 22 occurrences
François Bayrou : 21 occurrences
Europe : 19 occurrences
Russie : 18 occurrences
franceinfo : 16 occurrences
Français : 16 occurrences
TikTok : 15 occurrences
cyclone Chido : 14 occurrences
Premier ministre : 13 occurrences
Syrie : 13 occurrences
la France : 13 occurrences
État : 11 occurrences
Moscou : 11 occurrences
Balkans : 11 occurrences
Action : 11 occurrences
Commission européenne : 9 occurrences


In [73]:

# Afficher les entités par article
print("\nEntités extraites par article :")
print()
for link, data in resultats.items():
    print(f"\nArticle : {link}")
    for ent in data['entities']:
        print(f" - Texte: {ent['text']}, Type: {ent['type']}, Span: ({ent['start']}, {ent['end']})")


Entités extraites par article :


Article : https://www.francetvinfo.fr/replay-radio/aujourd-hui-c-est-demain/cyclone-chido-a-mayotte-les-alternatives-pour-produire-de-l-electricite-dans-les-zones-devastees_6934427.html#xtor=RSS-3-[general]
 - Texte: Mayotte, Type: LOC, Span: (2, 9)
 - Texte: cyclone Chido, Type: MISC, Span: (27, 40)
 - Texte: Mayotte, Type: LOC, Span: (347, 354)
 - Texte: cyclone Chido, Type: MISC, Span: (372, 385)
 - Texte: François-Noël Buffet, Type: PER, Span: (473, 493)
 - Texte: Shall Energy, Type: MISC, Span: (847, 859)
 - Texte: Portable, Type: MISC, Span: (907, 915)
 - Texte: Mayotte, Type: LOC, Span: (1574, 1581)

Article : https://www.francetvinfo.fr/faits-divers/les-affrontements-entre-bandes-sont-en-legere-hausse-dans-l-agglomeration-parisienne-selon-le-prefet-de-police_6962687.html#xtor=RSS-3-[general]
 - Texte: Paris, Type: LOC, Span: (131, 136)
 - Texte: RMC, Type: ORG, Span: (206, 209)
 - Texte: Laurent Nuñez, Type: PER, Span: (211, 224)
 - Texte: Sei

In [74]:
resultats

{'https://www.francetvinfo.fr/replay-radio/aujourd-hui-c-est-demain/cyclone-chido-a-mayotte-les-alternatives-pour-produire-de-l-electricite-dans-les-zones-devastees_6934427.html#xtor=RSS-3-[general]': {'entities': [{'text': 'Mayotte',
    'type': 'LOC',
    'start': 2,
    'end': 9},
   {'text': 'cyclone Chido', 'type': 'MISC', 'start': 27, 'end': 40},
   {'text': 'Mayotte', 'type': 'LOC', 'start': 347, 'end': 354},
   {'text': 'cyclone Chido', 'type': 'MISC', 'start': 372, 'end': 385},
   {'text': 'François-Noël Buffet', 'type': 'PER', 'start': 473, 'end': 493},
   {'text': 'Shall Energy', 'type': 'MISC', 'start': 847, 'end': 859},
   {'text': 'Portable', 'type': 'MISC', 'start': 907, 'end': 915},
   {'text': 'Mayotte', 'type': 'LOC', 'start': 1574, 'end': 1581}]},
 'https://www.francetvinfo.fr/faits-divers/les-affrontements-entre-bandes-sont-en-legere-hausse-dans-l-agglomeration-parisienne-selon-le-prefet-de-police_6962687.html#xtor=RSS-3-[general]': {'entities': [{'text': 'Paris',
 

In [75]:
from collections import Counter

# Initialiser les compteurs pour chaque type d'entité
person_counter = Counter()
org_counter = Counter()
loc_counter = Counter()

# Parcourir les résultats et compter les entités par type
for lien, contenu in resultats.items():
    for ent in contenu['entities']:
        if ent['type'] == 'PER':  # Personnes
            person_counter[ent['text']] += 1
        elif ent['type'] == 'ORG':  # Organisations
            org_counter[ent['text']] += 1
        elif ent['type'] == 'LOC':  # Lieux
            loc_counter[ent['text']] += 1

# Obtenir les 20 entités les plus fréquentes pour chaque type
top_persons = person_counter.most_common(20)
top_organizations = org_counter.most_common(20)
top_locations = loc_counter.most_common(20)

# Stocker les résultats dans des listes
persons_list = [ent for ent, freq in top_persons]
organizations_list = [ent for ent, freq in top_organizations]
locations_list = [ent for ent, freq in top_locations]

# Afficher les résultats
print("Top 20 des entités de type PERSON :")
for ent, freq in top_persons:
    print(f"{ent}: {freq}")

print("\nTop 20 des entités de type ORG :")
for ent, freq in top_organizations:
    print(f"{ent}: {freq}")

print("\nTop 20 des entités de type LOC :")
for ent, freq in top_locations:
    print(f"{ent}: {freq}")

# Résumé des listes
print("\nListes des entités les plus fréquentes :")
print(f"Personnes : {persons_list}")
print(f"Organisations : {organizations_list}")
print(f"Lieux : {locations_list}")


Top 20 des entités de type PERSON :
François Bayrou: 21
Emmanuel Macron: 8
Volodymyr Zelensky: 8
Anna Colin Lebedev: 7
Étienne Leenhardt: 7
Isabelle Patrier: 7
Philippe Crevel: 7
Donald Trump: 6
Ursula von der Leyen: 6
Elon Musk: 5
François Hollande: 4
François-Xavier Bellamy: 4
Trump: 4
Olaf Scholz: 4
Philippe Leclerc: 4
Jordan Bardella: 4
Cornille Havard: 4
Laurent Nuñez: 3
président de la République: 3
Paul D.: 3

Top 20 des entités de type ORG :
UE: 29
franceinfo: 16
TikTok: 15
Action: 11
Commission européenne: 9
Gaîté Lyrique: 7
Union européenne: 7
SNCF: 6
TotalEnergies: 6
Action France: 6
CSE: 5
Fret SNCF: 5
France 2: 4
Parlement: 4
Commission: 4
Otan: 4
Chinese Business Club: 4
Banque centrale européenne: 4
BCE: 4
Assemblée nationale: 3

Top 20 des entités de type LOC :
Mayotte: 37
Paris: 23
Ukraine: 22
France: 22
Russie: 18
Europe: 18
Français: 13
Syrie: 13
la France: 13
État: 11
Moscou: 11
Balkans: 11
Bruxelles: 9
Etat: 8
Etats-Unis: 7
Allemagne: 7
Japon: 7
Comores: 6
Matignon

In [76]:
top_persons 



[('François Bayrou', 21),
 ('Emmanuel Macron', 8),
 ('Volodymyr Zelensky', 8),
 ('Anna Colin Lebedev', 7),
 ('Étienne Leenhardt', 7),
 ('Isabelle Patrier', 7),
 ('Philippe Crevel', 7),
 ('Donald Trump', 6),
 ('Ursula von der Leyen', 6),
 ('Elon Musk', 5),
 ('François Hollande', 4),
 ('François-Xavier Bellamy', 4),
 ('Trump', 4),
 ('Olaf Scholz', 4),
 ('Philippe Leclerc', 4),
 ('Jordan Bardella', 4),
 ('Cornille Havard', 4),
 ('Laurent Nuñez', 3),
 ('président de la République', 3),
 ('Paul D.', 3)]

In [77]:
top_organizations 


[('UE', 29),
 ('franceinfo', 16),
 ('TikTok', 15),
 ('Action', 11),
 ('Commission européenne', 9),
 ('Gaîté Lyrique', 7),
 ('Union européenne', 7),
 ('SNCF', 6),
 ('TotalEnergies', 6),
 ('Action France', 6),
 ('CSE', 5),
 ('Fret SNCF', 5),
 ('France 2', 4),
 ('Parlement', 4),
 ('Commission', 4),
 ('Otan', 4),
 ('Chinese Business Club', 4),
 ('Banque centrale européenne', 4),
 ('BCE', 4),
 ('Assemblée nationale', 3)]

In [78]:
top_locations 

[('Mayotte', 37),
 ('Paris', 23),
 ('Ukraine', 22),
 ('France', 22),
 ('Russie', 18),
 ('Europe', 18),
 ('Français', 13),
 ('Syrie', 13),
 ('la France', 13),
 ('État', 11),
 ('Moscou', 11),
 ('Balkans', 11),
 ('Bruxelles', 9),
 ('Etat', 8),
 ('Etats-Unis', 7),
 ('Allemagne', 7),
 ('Japon', 7),
 ('Comores', 6),
 ('Matignon', 6),
 ('Kiev', 6)]

### Question 10

In [79]:
def compter_cooccurrences(collection, entite1, type1, entite2, type2):
    """
    Compte le nombre de co-occurrences de deux entités (avec leurs types) dans une collection de documents.

    Args:
        collection (dict): Un dictionnaire contenant les documents avec leurs entités.
        entite1 (str): La première entité.
        type1 (str): Le type de la première entité (ex: 'PER', 'ORG', 'LOC').
        entite2 (str): La deuxième entité.
        type2 (str): Le type de la deuxième entité (ex: 'PER', 'ORG', 'LOC').

    Returns:
        int: Le nombre de documents où les deux entités coexistent.
    """
    cooccurrence_count = 0

    # Parcourir chaque document dans la collection
    for doc_id, contenu in collection.items():
        entities_in_doc = contenu['entities']  # Liste des entités dans le document
        # Vérifier si les deux entités apparaissent dans le même document
        found_entite1 = any(ent['text'] == entite1 and ent['type'] == type1 for ent in entities_in_doc)
        found_entite2 = any(ent['text'] == entite2 and ent['type'] == type2 for ent in entities_in_doc)
        
        # Si les deux entités sont présentes, incrémenter le compteur
        if found_entite1 and found_entite2:
            cooccurrence_count += 1

    return cooccurrence_count


In [80]:
cooccurrences = compter_cooccurrences(resultats, 'François Bayrou', 'PER', 'Mayotte', 'LOC')

# Affichage du résultat
print(f"Nombre de co-occurrences entre 'François Bayrou' (PER) et 'Mayotte' (LOC) : {cooccurrences}")

Nombre de co-occurrences entre 'François Bayrou' (PER) et 'Mayotte' (LOC) : 4


### question 11

In [81]:
from itertools import combinations
# Stocker les résultats des entités dans des listes
persons_list = [ent for ent, _ in top_persons]
organizations_list = [ent for ent, _ in top_organizations]
locations_list = [ent for ent, _ in top_locations]

# Liste des paires d'entités à vérifier (en évitant les paires du même type)
pairs = []

# Créer des paires d'entités entre les différentes catégories, mais sans dupliquer le type
pairs += [(p, o) for p in persons_list for o in organizations_list]  # Paires entre personnes et organisations
pairs += [(p, l) for p in persons_list for l in locations_list]  # Paires entre personnes et lieux
pairs += [(o, l) for o in organizations_list for l in locations_list]  # Paires entre organisations et lieux

# Compter les co-occurrences pour chaque paire
cooccurrences_results = {}

for ent1, ent2 in pairs:
    type1 = 'PER' if ent1 in persons_list else 'ORG' if ent1 in organizations_list else 'LOC'
    type2 = 'PER' if ent2 in persons_list else 'ORG' if ent2 in organizations_list else 'LOC'
    cooccurrences_results[(ent1, ent2)] = compter_cooccurrences(resultats, ent1, type1, ent2, type2)

# Trier les résultats par le nombre de co-occurrences, en ordre décroissant
top_cooccurrences = Counter(cooccurrences_results).most_common(2)

# Affichage des deux paires les plus fréquentes
print("Les deux paires d'entités les plus fréquentes sont :")
for pair, count in top_cooccurrences:
    print(f"Paire : {pair}, Co-occurrences : {count}")

Les deux paires d'entités les plus fréquentes sont :
Paire : ('Volodymyr Zelensky', 'Ukraine'), Co-occurrences : 6
Paire : ('franceinfo', 'Mayotte'), Co-occurrences : 5


### Question 12